In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gloss2sent/gloss2sent.csv


In [4]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

2024-05-13 01:49:19.908646: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-13 01:49:19.908750: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-13 01:49:20.056943: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
data = pd.read_csv("/kaggle/input/gloss2sent/gloss2sent.csv")
data.head()

,Glosses,Sentences
0,loud dog barks excited quiet bird fence.,The loud dog barks excitedly at the quiet bird...
1,Happy cow grazes big green pasture.,"The happy cow grazes peacefully in the big, gr..."
2,Man short hat shields eyes bright sunlight.,He wears a short hat to shield his eyes from t...
3,Girl New cell phone explored features.,"She excitedly shows off her new cell phone, ea..."
4,Fan cools hot summer day.,The fan hums softly as it cools the room on a ...


In [6]:
data.head()

,Glosses,Sentences
0,loud dog barks excited quiet bird fence.,The loud dog barks excitedly at the quiet bird...
1,Happy cow grazes big green pasture.,"The happy cow grazes peacefully in the big, gr..."
2,Man short hat shields eyes bright sunlight.,He wears a short hat to shield his eyes from t...
3,Girl New cell phone explored features.,"She excitedly shows off her new cell phone, ea..."
4,Fan cools hot summer day.,The fan hums softly as it cools the room on a ...


In [14]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer

In [15]:
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

In [16]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer

# Splitting the data into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Loading pre-trained tokenizer (e.g., BERT tokenizer)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Defining a custom PyTorch Dataset class
class SignLanguageDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        gloss = self.data.iloc[0]["Glosses"]
        sentence = self.data.iloc[0]["Sentences"]

        # Tokenize gloss and sentence
        inputs = self.tokenizer(gloss, sentence, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
        
        input_ids = inputs["input_ids"].squeeze(0)
        
        attention_mask = inputs["attention_mask"].squeeze(0)
        

        return {"input_ids": input_ids, "attention_mask": attention_mask}

# Creating DataLoader objects for training and validation sets
train_dataset = SignLanguageDataset(train_data, tokenizer)
val_dataset = SignLanguageDataset(val_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)


In [17]:
#student model 
import torch.nn as nn
from transformers import BertModel
class StudentModel(nn.Module):
    def __init__(self):
        super(StudentModel, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.fc = nn.Linear(self.bert.config.hidden_size, len(tokenizer.vocab))
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.fc(outputs.last_hidden_state[:, 0, :])
        return logits
student_model = StudentModel()

In [23]:
import torch.optim as optim
import torch.nn.functional as F

# Definin the temperature for knowledge distillation
temperature = 5.0  #adjustment based on preference


criterion = nn.KLDivLoss() #defining loss function
optimizer = optim.AdamW(student_model.parameters(), lr=1e-5) # defining optimizer

# Training loop
for epoch in range(7): #can be adjusted as per requirements
    student_model.train()
    for batch in train_loader:
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]

        # Forward pass: student model
        logits_student = student_model(input_ids, attention_mask)

        # Computing the knowledge distillation loss
        kd_loss = criterion(F.log_softmax(logits_student / temperature, dim=-1),
                            F.softmax(logits_student / temperature, dim=-1))

        # Backpropagation and optimization
        optimizer.zero_grad()
        kd_loss.backward()
        optimizer.step()

    # Validation loop (evaluating the student model performance on validation set)
    student_model.eval()
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"]
            attention_mask = batch["attention_mask"]

            # Forward pass
            logits_student = student_model(input_ids, attention_mask)

          # Next up, we can compute the validation metrics
        validation_loss = compute_validation_loss(model, criterion, val_loader)
        
   """
   def compute_validation_loss(student_model, criterion, data_loader):
            model.eval()
            val_loss = 0.0
            num_samples = 0
            with torch.no_grad():
            for batch in data_loader:
                input_ids = batch["input_ids"]
                attention_mask = batch["attention_mask"]
                logits_student = model(input_ids, attention_mask)
                labels = input_ids  # Assuming labels are the input_ids
                batch_loss = criterion(logits_student, labels)
                val_loss += batch_loss.item() * input_ids.size(0)
                num_samples += input_ids.size(0)
            return val_loss / num_samples

# Printing the result
print("Validation Loss:", validation_loss)
"""


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 43)